# Train 
The network can be trained using three interfaces:

- python, via `das.train.train`
- the command-line interface `das train`.
- the GUI - see the [GUI tutorial](/tutorials_gui/train)

Training will:

- load train/val/test data form a dataset
- initialize the network
- save all parameters for reproducibility
- train the network and save the best network to disk
- run inference and evaluate the network using the test data.

The names of files created during training start with an optional prefix and the time stamp of the start time of training, as in `my-awesome-prefix_20192310_091032`. Typically, three files are created:
- `*_params.yaml` - training parameters etc.
- `*_model.h5` -  model architecture and weights
- `*_results.h5` - predictions and evaluation results for the test set (only created if the training dataset contains a test set)

## Training using python
Training is done using the `train` function in the `das.train` module:

In [1]:
import das.train
help(das.train.train)

Help on function train in module das.train:

train(*, data_dir: str, y_suffix: str = '', save_dir: str = './', save_prefix: Union[str, NoneType] = None, model_name: str = 'tcn', nb_filters: int = 16, kernel_size: int = 16, nb_conv: int = 3, use_separable: List[bool] = False, nb_hist: int = 1024, ignore_boundaries: bool = True, batch_norm: bool = True, nb_pre_conv: int = 0, pre_nb_dft: int = 64, pre_kernel_size: int = 3, pre_nb_filters: int = 16, pre_nb_conv: int = 2, nb_lstm_units: int = 0, verbose: int = 2, batch_size: int = 32, nb_epoch: int = 400, learning_rate: Union[float, NoneType] = None, reduce_lr: bool = False, reduce_lr_patience: int = 5, fraction_data: Union[float, NoneType] = None, seed: Union[int, NoneType] = None, batch_level_subsampling: bool = False, tensorboard: bool = False, neptune_api_token: Union[str, NoneType] = None, neptune_project: Union[str, NoneType] = None, log_messages: bool = False, nb_stacks: int = 2, with_y_hist: bool = True, x_suffix: str = '', balance:

Calling the `train` function produce fairly verbose logging messages, to help with troubleshooting:

- run time parameters
- information on the size of the training and validation data
- network architecture
- training progress (training and validation loss)
- after training, a classification report for the test data (if test data exist in the dataset)

When done, `train` returns the trained keras model and a parameter dictionary with all arguments required to reproduce the model.

To demonstrate the outputs of `train`, the following trains a small network on a small dataset to annotate pulse and sine song from _Drosophila melanogater_. Expected performance (f1-score) is about 75%.

In [2]:
model, params = das.train.train(model_name='tcn',  # see `das.models` for valid model_names
                                data_dir='tutorial_dataset.npy', 
                                save_dir='res',
                                nb_hist=256,
                                kernel_size=16,
                                nb_filters=16,
                                ignore_boundaries=True,
                                verbose=1,
                                nb_epoch=4,
                                log_messages=True)

INFO:root:Loading data from tutorial_dataset.npy.
INFO:root:Version of the data:
INFO:root:   MD5 hash of tutorial_dataset.npy is
INFO:root:   34876fb30412a444e444a8e1f5312126
INFO:root:Parameters:
INFO:root:{'data_dir': 'tutorial_dataset.npy', 'y_suffix': '', 'save_dir': 'res', 'save_prefix': '', 'model_name': 'tcn', 'nb_filters': 16, 'kernel_size': 16, 'nb_conv': 3, 'use_separable': False, 'nb_hist': 256, 'ignore_boundaries': True, 'batch_norm': True, 'nb_pre_conv': 0, 'pre_nb_dft': 64, 'pre_kernel_size': 3, 'pre_nb_filters': 16, 'pre_nb_conv': 2, 'nb_lstm_units': 0, 'verbose': 1, 'batch_size': 32, 'nb_epoch': 4, 'reduce_lr': False, 'reduce_lr_patience': 5, 'fraction_data': None, 'seed': None, 'batch_level_subsampling': False, 'tensorboard': False, 'neptune_api_token': None, 'neptune_project': None, 'log_messages': True, 'nb_stacks': 2, 'with_y_hist': True, 'x_suffix': '', 'balance': False, 'version_data': True, 'sample_weight_mode': 'temporal', 'data_padding': 48, 'return_sequences'

Model: "TCN"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 1)]     0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 256, 16)      32          input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 256, 16)      4112        conv1d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 256, 16)      0           conv1d_1[0][0]                   
________________________________________________________________________________________________

/Users/janc/miniconda3/lib/python3.8/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '



Epoch 00001: val_loss improved from inf to 0.11043, saving model to res/20210924_220702_model.h5


/Users/janc/miniconda3/lib/python3.8/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


1000/1000 [==============================] - 241s 236ms/step - batch: 499.5000 - size: 32.0000 - loss: 0.1143 - val_loss: 0.1104
Epoch 2/4
1000/1000 [==============================] - ETA: 0s - batch: 499.5000 - size: 32.0000 - loss: 0.0841
Epoch 00002: val_loss improved from 0.11043 to 0.10770, saving model to res/20210924_220702_model.h5
1000/1000 [==============================] - 226s 226ms/step - batch: 499.5000 - size: 32.0000 - loss: 0.0841 - val_loss: 0.1077
Epoch 3/4
1000/1000 [==============================] - ETA: 0s - batch: 499.5000 - size: 32.0000 - loss: 0.0867
Epoch 00003: val_loss did not improve from 0.10770
1000/1000 [==============================] - 215s 215ms/step - batch: 499.5000 - size: 32.0000 - loss: 0.0867 - val_loss: 0.1103
Epoch 4/4
1000/1000 [==============================] - ETA: 0s - batch: 499.5000 - size: 32.0000 - loss: 0.0823
Epoch 00004: val_loss improved from 0.10770 to 0.09988, saving model to res/20210924_220702_model.h5
1000/1000 [=============

INFO:root:re-loading last best model
INFO:root:predicting
INFO:root:evaluating
INFO:root:[[3545939    7799   38820]
 [  10658   33510     140]
 [  99569      58  241747]]
INFO:root:{'noise': {'precision': 0.9698517518077681, 'recall': 0.9870234523701497, 'f1-score': 0.9783622607234045, 'support': 3592558}, 'pulse': {'precision': 0.8100659946334035, 'recall': 0.7562968312720051, 'f1-score': 0.7822585351619492, 'support': 44308}, 'sine': {'precision': 0.8612075936830931, 'recall': 0.7081587935812335, 'f1-score': 0.7772203298284307, 'support': 341374}, 'accuracy': 0.960524251930502, 'macro avg': {'precision': 0.8803751133747548, 'recall': 0.8171596924077962, 'f1-score': 0.8459470419045948, 'support': 3978240}, 'weighted avg': {'precision': 0.9587493351198522, 'recall': 0.960524251930502, 'f1-score': 0.958918087071358, 'support': 3978240}}
INFO:root:saving to res/20210924_220702_results.h5.
/Users/janc/miniconda3/lib/python3.8/site-packages/tables/attributeset.py:464: NaturalNameWarning: o

## Training using command-line scripts
The training function `das.train.train` and all its arguments are also accessible from the command line via `das train` for use on the terminal. See [here](/technical/cli#train) for a description of all command-line arguments. The command-line interface is generated with [defopt](https://defopt.readthedocs.io/en/stable/index.html).

For instance, training command above can be invoked from the command line:
```shell
das train --data-dir dat/dmel_single_raw.npy --save-dir res --model-name tcn --kernel-size 16 --nb-filters 16 --nb-hist 512 --nb-epoch 20 -i
```

Shell scripts are particularly useful if you want to fit the network with with different configurations to optimize [structural parameters](/tutorials/structparams). For instance, this script will fit networks with different numbers of TCN blocks (`nb_conv`) and filters (`nb_filters`):
```shell
#!/bin/bash
conda activate das

YSUFFIX="pulse"
MODELNAME='tcn'
DATADIR='../dat/dmel_single.npy'
SAVEDIR="res"

NB_HIST=2048
KERNEL_SIZE=32
NB_FILTERS=32
NB_CONV=3

for NB_CONV in 2 3 4
do
    for NB_FILTERS in 16 32 64
    do
        das train -i --nb-filters $NB_FILTERS --kernel-size $KERNEL_SIZE --nb-conv $NB_CONV --nb-hist $NB_HIST --save-dir $SAVEDIR --y-suffix $YSUFFIX --data-dir $DATADIR --model-name $MODELNAME
    done
done
```

A description of all command line arguments can be obtained by typing `das train --help` in a terminal:

In [3]:
!das train --help

usage: das train [-h] -d DATA_DIR [-y Y_SUFFIX] [--save-dir SAVE_DIR]
                 [--save-prefix SAVE_PREFIX] [-m MODEL_NAME]
                 [--nb-filters NB_FILTERS] [-k KERNEL_SIZE]
                 [--nb-conv NB_CONV] [-u [USE_SEPARABLE [USE_SEPARABLE ...]]]
                 [--nb-hist NB_HIST]
                 [-i | --ignore-boundaries | --no-ignore-boundaries]
                 [--batch-norm | --no-batch-norm] [--nb-pre-conv NB_PRE_CONV]
                 [--pre-nb-dft PRE_NB_DFT] [--pre-kernel-size PRE_KERNEL_SIZE]
                 [--pre-nb-filters PRE_NB_FILTERS] [--pre-nb-conv PRE_NB_CONV]
                 [--nb-lstm-units NB_LSTM_UNITS] [--verbose VERBOSE]
                 [--batch-size BATCH_SIZE] [--nb-epoch NB_EPOCH]
                 [--learning-rate LEARNING_RATE]
                 [--reduce-lr | --no-reduce-lr]
                 [--reduce-lr-patience REDUCE_LR_PATIENCE] [-f FRACTION_DATA]
                 [--seed SEED]
                 [--batch-level-subsampling | --n